In [141]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

pl2 =  pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3401_20200720_0916_day\Logs_full\mdLog_fut2_20200720_0916.csv')
nl2 = pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3402_20200720_0954_day\Logs_full\mdLog_fut2_20200720_0954.csv')
pl1 =  pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3401_20200720_0916_day\Logs_full\mdLog_fut_20200720_0916.csv')
nl1 = pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3402_20200720_0954_day\Logs_full\mdLog_fut_20200720_0954.csv')
pl2 = pd.merge(pl2, pl1[['eventId', 'exchangeTime', 'type']], on='eventId', how='outer')
assert((pl2[pl2['exchangeTime'].isnull()].shape[0] == 0) & (pl2[pl2['secid'].isnull()].shape[0] == 0))
nl2 = pd.merge(nl2, nl1[['eventId', 'exchangeTime', 'type']], on='eventId', how='outer')
assert((pl2[pl2['exchangeTime'].isnull()].shape[0] == 0) & (pl2[pl2['secid'].isnull()].shape[0] == 0))
pl2['time'] = pl2['ms'].apply(lambda x: x.split('.')[0])
pl2['time'] = pl2['time'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1] + x.split(':')[2]))
nl2['time'] = nl2['ms'].apply(lambda x: x.split('.')[0])
nl2['time'] = nl2['time'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1] + x.split(':')[2]))

In [174]:
pl2

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,exchangeTime,type,time
0,09:29:32.526412,1595208572527258,1595208572527166,1,2158,IC2009,1,6318.0,126,6313.0,6313.2,6313.6,6314.4,6315.2,6318.0,6319.0,6319.6,6319.8,6320.0,1,10,1,1,2,27,50,1,1,4,NaN,65563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.590455e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:30:00|500,1,92932
1,09:29:32.526412,1595208572527279,1595208572527199,2,2158,IC2009,1,6318.0,126,6313.0,6313.2,6313.6,6314.4,6315.2,6318.0,6319.0,6319.6,6319.8,6320.0,1,10,1,1,2,27,50,1,1,4,NaN,65563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.590455e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:30:00|500,3,92932
2,09:29:32.526412,1595208572527296,1595208572527208,3,2158,IF2009,1,4552.0,87,4538.8,4540.0,4550.0,4552.0,4552.2,4552.6,4554.4,4554.8,4555.0,4555.2,1,4,12,9,3,1,4,3,4,1,NaN,58723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.188016e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:30:00|500,3,92932
3,09:29:32.526412,1595208572527312,1595208572527211,4,2158,IF2009,1,4552.0,87,4538.8,4540.0,4550.0,4552.0,4552.2,4552.6,4554.4,4554.8,4555.0,4555.2,1,4,12,9,3,1,4,3,4,1,NaN,58723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.188016e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:30:00|500,1,92932
4,09:29:32.526412,1595208572527425,1595208572527400,5,2158,IH2009,1,3215.4,74,3207.4,3208.0,3208.2,3210.0,3214.8,3215.2,3215.4,3216.0,3218.0,3218.6,4,1,1,3,6,2,21,2,8,1,NaN,24847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.137708e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:30:00|500,1,92932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102273,15:31:52.332713,1595230312333472,1595230312333441,1102274,2158,IF2103,1,4567.6,685,4532.2,4537.6,4550.0,4555.0,4555.6,4569.8,4608.8,4646.0,4660.0,4662.4,1,1,1,1,1,1,1,3,1,1,NaN,641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.287669e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:32:19|500,0,153152
1102274,15:31:52.332713,1595230312333487,1595230312333444,1102275,2158,IH2103,1,3255.8,437,3200.0,3208.0,3216.0,3232.4,3250.0,3262.8,3269.0,3269.6,3275.0,3300.0,1,1,1,1,2,1,1,1,1,1,NaN,407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.226626e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:32:19|500,0,153152
1102275,15:31:52.333713,1595230312334098,1595230312334074,1102276,2158,IC2103,1,6105.0,2540,6059.0,6060.6,6090.2,6091.2,6096.2,6114.0,6114.6,6115.0,6117.2,6119.8,1,1,3,2,2,1,8,1,1,1,NaN,2407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.076148e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:32:19|500,2,153152
1102276,15:31:52.333713,1595230312334133,1595230312334111,1102277,2158,IF2103,1,4567.6,685,4532.2,4537.6,4550.0,4555.0,4555.6,4569.8,4608.8,4646.0,4660.0,4662.4,1,1,1,1,1,1,1,3,1,1,NaN,641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.287669e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:32:19|500,2,153152


In [140]:
pl1[pl1["eventId"] == 125296]

,ms,clock,clockAtArrival,eventId,threadId,event,secid,lastPrice,lastSize,volume,bidSize,bid,ask,askSize,note,isMD,state,tick,status,oi,stale,mdType,turnover,exchangeTime,type,priceChangeQueueCounter,priceChangeNoVolumeQueueCounter
125295,09:54:10.547296,1595210050547644,1595210050547615,125296,2158,NaN,IF2009,4518.4,0,6484,1,4516.2,4519.6,1,NaN,1,NaN,NaN,NaN,56461,NaN,NaN,8.830318e+09,09:54:38|500,3,NaN,NaN


In [142]:
columns = ["secid", "lastPrice", "volume", "buyP5", "buyP4", "buyP3", "buyP2", "buyP1", "sellP1", "sellP2",
           "sellP3", "sellP4", "sellP5", "buyQ5", "buyQ4", "buyQ3", "buyQ2", "buyQ1", "sellQ1", "sellQ2", "sellQ3",
           "sellQ4", "sellQ5", 'turnover', 'exchangeTime']
data1_1 = pl2.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = nl2.drop_duplicates(subset=columns, keep="first").reset_index()
data1_1 = data1_1[data1_1['exchangeTime'] >= '09:54:38|500']
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["clock_x"].count()
n2 = test["clock_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)

250146
250448
250448


In [169]:
test[test['clock_x'].isnull()].head(100)

,index_x,ms_x,clock_x,clockAtArrival_x,eventId_x,threadId_x,secid,isFaster_x,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime_x,oi_x,suspension_x,buyQI5_x,buyQI4_x,buyQI3_x,buyQI2_x,buyQI1_x,sellQI1_x,sellQI2_x,sellQI3_x,sellQI4_x,sellQI5_x,tick_x,turnover,temp_x,buyP10_x,buyP9_x,buyP8_x,buyP7_x,buyP6_x,sellP6_x,sellP7_x,sellP8_x,sellP9_x,sellP10_x,buyQ10_x,buyQ9_x,buyQ8_x,buyQ7_x,buyQ6_x,sellQ6_x,sellQ7_x,sellQ8_x,sellQ9_x,sellQ10_x,exchangeTime,type_x,time_x,index_y,ms_y,clock_y,clockAtArrival_y,eventId_y,threadId_y,isFaster_y,genTime_y,oi_y,suspension_y,buyQI5_y,buyQI4_y,buyQI3_y,buyQI2_y,buyQI1_y,sellQI1_y,sellQI2_y,sellQI3_y,sellQI4_y,sellQI5_y,tick_y,temp_y,buyP10_y,buyP9_y,buyP8_y,buyP7_y,buyP6_y,sellP6_y,sellP7_y,sellP8_y,sellP9_y,sellP10_y,buyQ10_y,buyQ9_y,buyQ8_y,buyQ7_y,buyQ6_y,sellQ6_y,sellQ7_y,sellQ8_y,sellQ9_y,sellQ10_y,type_y,time_y
250146,NaN,NaN,NaN,NaN,NaN,NaN,IH2008,NaN,3190.8,10529,3189.2,3189.4,3189.6,3189.8,3190.0,3190.6,3190.8,3191.0,3191.2,3191.8,1,1,1,1,2,1,5,2,1,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.012572e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:58:26|0,NaN,NaN,18228,09:58:33.585487,1595210313586170,1595210313586149,18229,32540,1,NaN,32397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,95833
250147,NaN,NaN,NaN,NaN,NaN,NaN,IC2012,NaN,6107.2,3710,6099.0,6100.0,6100.8,6101.0,6101.2,6107.2,6107.4,6107.8,6108.0,6109.0,2,7,1,3,1,4,1,2,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.561246e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:59:04|0,NaN,NaN,21480,09:59:11.597484,1595210351598643,1595210351598602,21481,32540,1,NaN,47819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,95911
250148,NaN,NaN,NaN,NaN,NaN,NaN,IF2012,NaN,4475.0,2117,4471.8,4472.0,4472.8,4473.0,4474.4,4475.8,4477.0,4477.2,4477.8,4478.2,1,2,1,1,2,1,1,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.862159e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:59:04|0,NaN,NaN,21481,09:59:11.597484,1595210351598690,1595210351598649,21482,32540,1,NaN,26272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,95911
250149,NaN,NaN,NaN,NaN,NaN,NaN,IH2012,NaN,3170.2,1514,3165.0,3165.2,3165.6,3166.4,3166.8,3170.0,3170.2,3171.8,3173.6,3173.8,8,1,1,1,1,1,1,1,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.447891e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:59:04|0,NaN,NaN,21482,09:59:11.597484,1595210351598749,1595210351598706,21483,32540,1,NaN,13991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,95911
250150,NaN,NaN,NaN,NaN,NaN,NaN,IF2012,NaN,4478.8,2373,4475.0,4476.8,4477.2,4477.6,4477.8,4478.8,4479.6,4482.4,4482.6,4484.4,2,1,2,1,1,9,4,9,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.205894e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:02:59|500,NaN,NaN,40092,10:03:07.178463,1595210587179208,1595210587179174,40093,32540,1,NaN,26294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,100307
250151,NaN,NaN,NaN,NaN,NaN,NaN,IC2008,NaN,6322.0,26007,6319.8,6320.0,6320.2,6320.4,6321.8,6322.4,6323.0,6323.4,6323.6,6324.4,3,23,6,2,12,1,9,2,2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.306030e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:02:59|500,NaN,NaN,40096,10:03:07.178463,159521

In [173]:
pl2[(pl2['secid'] == 'IF2008') & (pl2['exchangeTime'] >= '10:07:12|500')].head(12)[['ms', 'clockAtArrival', 'secid', 'lastPrice', 'volume', 'type', 'exchangeTime']]

,ms,clockAtArrival,secid,lastPrice,volume,type,exchangeTime
182790,10:06:44.569237,1595210804570565,IF2008,4542.0,30645,1,10:07:12|500
182793,10:06:44.569237,1595210804570583,IF2008,4542.0,30645,3,10:07:12|500
182796,10:06:44.569237,1595210804570595,IF2008,4542.0,30645,0,10:07:12|500
182799,10:06:44.569237,1595210804570918,IF2008,4542.0,30645,2,10:07:12|500
182846,10:06:45.571237,1595210805572352,IF2008,4541.8,30661,1,10:07:13|500
182849,10:06:45.571237,1595210805572378,IF2008,4541.8,30661,3,10:07:13|500
182855,10:06:45.571237,1595210805572774,IF2008,4541.8,30661,0,10:07:13|500
182858,10:06:45.571237,1595210805572911,IF2008,4541.8,30661,2,10:07:13|500
182892,10:06:46.071237,1595210806072949,IF2008,4541.6,30665,1,10:07:14|0
182897,10:06:46.072237,1595210806073001,IF2008,4541.6,30665,3,10:07:14|0


In [172]:
nl2[(nl2['secid'] == 'IF2008') & (nl2['exchangeTime'] >= '10:07:12|500')].head(12)[['ms', 'clockAtArrival', 'secid', 'lastPrice', 'volume', 'type', 'exchangeTime']]

,ms,clockAtArrival,secid,lastPrice,volume,type,exchangeTime
58501,10:07:20.110440,1595210840111039,IF2008,4542.0,30645,1,10:07:12|500
58504,10:07:20.110440,1595210840111159,IF2008,4542.0,30645,3,10:07:12|500
58507,10:07:20.110440,1595210840111243,IF2008,4542.0,30645,0,10:07:12|500
58510,10:07:20.110440,1595210840111361,IF2008,4542.0,30645,2,10:07:12|500
58538,10:07:20.613440,1595210840614339,IF2008,4542.2,30652,1,10:07:13|0
58541,10:07:20.613440,1595210840614440,IF2008,4542.2,30652,3,10:07:13|0
58547,10:07:20.613440,1595210840614726,IF2008,4542.2,30652,0,10:07:13|0
58550,10:07:20.614440,1595210840614892,IF2008,4542.2,30652,2,10:07:13|0
58583,10:07:21.112440,1595210841113294,IF2008,4541.8,30661,1,10:07:13|500
58587,10:07:21.112440,1595210841113435,IF2008,4541.8,30661,3,10:07:13|500


In [159]:
pl1[(pl1['secid'] == 'IH2008') & (pl1['eventId'] >= 143195)]

,ms,clock,clockAtArrival,eventId,threadId,event,secid,lastPrice,lastSize,volume,bidSize,bid,ask,askSize,note,isMD,state,tick,status,oi,stale,mdType,turnover,exchangeTime,type,priceChangeQueueCounter,priceChangeNoVolumeQueueCounter
143194,09:57:57.547279,1595210277548380,1595210277548340,143195,2158,NaN,IH2008,3190.6,0,10528,1,3189.8,3190.8,6,NaN,1,NaN,NaN,NaN,32397,NaN,NaN,1.012477e+10,09:58:25|500,1,NaN,NaN
143195,09:57:57.547279,1595210277548398,1595210277548348,143196,2158,NaN,IH2008,3190.6,0,10528,1,3189.8,3190.8,6,NaN,1,NaN,NaN,NaN,32397,NaN,NaN,1.012477e+10,09:58:25|500,3,NaN,NaN
143200,09:57:57.548279,1595210277548576,1595210277548549,143201,2158,NaN,IH2008,3190.6,0,10528,1,3189.8,3190.8,6,NaN,1,NaN,NaN,NaN,32397,NaN,NaN,1.012477e+10,09:58:25|500,0,NaN,NaN
143203,09:57:57.548279,1595210277548649,1595210277548632,143204,2158,NaN,IH2008,3190.6,0,10528,1,3189.8,3190.8,6,NaN,1,NaN,NaN,NaN,32397,NaN,NaN,1.012477e+10,09:58:25|500,2,NaN,NaN
143260,09:57:58.552279,1595210278553207,1595210278552516,143261,2158,NaN,IH2008,3190.0,0,10531,1,3190.0,3190.8,4,NaN,1,NaN,NaN,NaN,32396,NaN,NaN,1.012764e+10,09:58:26|500,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102217,14:59:32.795864,1595228372796224,1595228372796199,1102218,2158,NaN,IH2008,3277.4,0,54361,2,3276.0,3277.2,1,NaN,1,NaN,NaN,NaN,39406,NaN,NaN,5.296482e+10,15:00:00|0,2,NaN,NaN
1102256,15:31:52.329713,1595230312330214,1595230312330193,1102257,2158,NaN,IH2008,3277.4,0,54361,2,3276.0,3277.2,1,NaN,1,NaN,NaN,NaN,39406,NaN,NaN,5.296482e+10,15:32:19|500,1,NaN,NaN
1102259,15:31:52.329713,1595230312330263,1595230312330217,1102260,2158,NaN,IH2008,3277.4,0,54361,2,3276.0,3277.2,1,NaN,1,NaN,NaN,NaN,39406,NaN,NaN,5.296482e+10,15:32:19|500,3,NaN,NaN
1102262,15:31:52.329713,1595230312330475,1595230312330463,1102263,2158,NaN,IH2008,3277.4,0,54361,2,3276.0,3277.2,1,NaN,1,NaN,NaN,NaN,39406,NaN,NaN,5.296482e+10,15:32:19|500,0,NaN,NaN


In [131]:
test[test['clock_x'].isnull()].groupby('secid')['time_y'].describe().fillna(0).astype('int64').sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
secid,,,,,,,,
IC2008,64,121694,17616,100307,103022,130175,140161,145648
IF2008,61,120145,17638,100307,102057,112031,135527,145648
IH2008,59,120862,18232,95833,102497,112056,137876,145648
IC2103,27,117715,15897,100307,104286,111435,133439,145006
IF2103,18,118810,17068,100636,102687,112038,133888,145006
IC2012,17,115047,17709,95911,101338,110750,135334,143326
IF2012,17,116640,19045,95911,100825,110750,140715,143326
IH2103,17,115623,14557,100307,104555,111435,132541,145006
IH2012,16,119958,18398,95911,103430,110920,136679,145648


In [132]:
# speed check
columns = ["secid", "lastPrice", "volume", "buyP5", "buyP4", "buyP3", "buyP2", "buyP1", "sellP1", "sellP2",
           "sellP3", "sellP4", "sellP5", "buyQ5", "buyQ4", "buyQ3", "buyQ2", "buyQ1", "sellQ1", "sellQ2", "sellQ3",
           "sellQ4", "sellQ5", 'turnover', 'exchangeTime']
pl2['count'] = pl2.groupby(['secid', 'exchangeTime'])['clock'].transform('nunique')
nl2['count'] = nl2.groupby(['secid', 'exchangeTime'])['clock'].transform('nunique')
data1 = pl2[pl2['count'] == 4]
data2 = nl2[nl2['count'] == 4]
# data2_1 = nl2.groupby(['secid', 'exchangeTime']).nth(0).reset_index()
# data1_1 = pl2.groupby(['secid', 'exchangeTime']).nth(0).reset_index()
# data1_1 = data1_1[data1_1['exchangeTime'] >= '09:54:38|500']
data1_1 = data1.drop_duplicates(subset=columns, keep="last").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="last").reset_index()
data1_1 = data1_1[data1_1['exchangeTime'] >= '09:54:38|500']
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["clock_x"].count()
n2 = test["clock_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
test1 = test[~np.isnan(test["clock_x"])]
test1 = test1[~np.isnan(test1["clock_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
# print(np.quantile(abs(diff[diff < 0]), 0.25))
# print(np.median(abs(diff[diff < 0])))
# print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
diff.describe([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')

233869
250448
250448
35564034.0
35646080.0
35677418.0
0.0
1.0


count      233869
mean     35623378
std         61607
min      35504768
1%       35534157
5%       35538960
10%      35545452
20%      35558181
30%      35569974
40%      35581867
50%      35646080
60%      35658181
70%      35670838
80%      35684186
90%      35697313
95%      35703872
99%      35708907
max      37532714
dtype: int64

In [145]:
pl2[(pl2['secid'] == 'IF2009') & (pl2['exchangeTime'] == '09:54:38|500')][['ms', 'clockAtArrival', 'secid', 'lastPrice', 'volume', 'type', 'exchangeTime']]

,ms,clockAtArrival,secid,lastPrice,volume,type,exchangeTime
125295,09:54:10.547296,1595210050547615,IF2009,4518.4,6484,3,09:54:38|500
125298,09:54:10.547296,1595210050547997,IF2009,4518.4,6484,1,09:54:38|500
125300,09:54:10.547296,1595210050548018,IF2009,4518.4,6484,0,09:54:38|500
125303,09:54:10.547296,1595210050548045,IF2009,4518.4,6484,2,09:54:38|500


In [144]:
nl2[(nl2['secid'] == 'IF2009') & (nl2['exchangeTime'] == '09:54:38|500')][['ms', 'clockAtArrival', 'secid', 'lastPrice', 'volume', 'type','exchangeTime']]

,ms,clockAtArrival,secid,lastPrice,volume,type,exchangeTime
1,09:54:48.082507,1595210088080618,IF2009,4518.4,6484,3,09:54:38|500
4,09:54:48.082507,1595210088080675,IF2009,4518.4,6484,1,09:54:38|500
10,09:54:48.082507,1595210088080729,IF2009,4518.4,6484,0,09:54:38|500
13,09:54:48.082507,1595210088080759,IF2009,4518.4,6484,2,09:54:38|500


In [95]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

nl2 =  pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3401_20200720_0916_day\Logs_full\mdLog_fut2_20200720_0916.csv')
pl2 = pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdl2BestAndDeepLog_20200720_1209.csv')
nl1 =  pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_fut_fn3401_20200720_0916_day\Logs_full\mdLog_fut_20200720_0916.csv')
pl1 = pd.read_csv(r'\\192.168.10.28\DataTeamShared\20200720\mdLog_20200720_1209.csv')
pl2 = pl2.iloc[1:, :]
corrections = {1088: 'IF2007',    
               1089: 'IF2008', 
               1090: 'IF2009', 
               1093: 'IF2012', 
               3363: 'IH2007',    
               3364: 'IH2008', 
               3365: 'IH2009', 
               3368: 'IH2012', 
               3463: 'IC2007',    
               3464: 'IC2008', 
               3465: 'IC2009', 
               3468: 'IC2012',  
               6263: 'XC2007',    
               6264: 'XC2008', 
               6265: 'XC2009', 
               6268: 'XC2012'}
pl2.secid = pl2.secid.map(corrections)
pl1.secid = pl1.secid.map(corrections)
pl2 = pl2[~pl2['secid'].isnull()]
pl1 = pl1[~pl1['secid'].isnull()]
pl1 = pl1.drop_duplicates(['clockAtArrival', 'exchangeTime', 'secid', 'threadId'])
display(pl2.shape[0])
pl2 = pd.merge(pl2, pl1[['clockAtArrival', 'exchangeTime', 'secid', 'threadId']], on=['clockAtArrival', 'secid', 'threadId'], how='left')
display(pl2.shape[0])
assert(pl2[pl2['secid'].isnull()].shape[0] == 0)
nl2 = pd.merge(nl2, nl1[['eventId', 'exchangeTime', 'type']], on='eventId', how='outer')
assert((pl2[pl2['exchangeTime'].isnull()].shape[0] == 0) & (pl2[pl2['secid'].isnull()].shape[0] == 0))
pl2['time'] = pl2['ms'].apply(lambda x: x.split(' ')[1].split('.')[0])
pl2['time'] = pl2['time'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1] + x.split(':')[2]))
nl2['time'] = nl2['ms'].apply(lambda x: x.split('.')[0])
nl2['time'] = nl2['time'].apply(lambda x: int(x.split(':')[0] + x.split(':')[1] + x.split(':')[2]))
nl2 = nl2[nl2['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
pl2 = pl2[pl2['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]

154077

154077

In [127]:
columns = ["secid", "lastPrice", "volume", "buyP5", "buyP4", "buyP3", "buyP2", "buyP1", "sellP1", "sellP2",
           "sellP3", "sellP4", "sellP5", "buyQ5", "buyQ4", "buyQ3", "buyQ2", "buyQ1", "sellQ1", "sellQ2", "sellQ3",
           "sellQ4", "sellQ5", 'turnover', 'exchangeTime']
pl2['count'] = pl2.groupby(['secid', 'exchangeTime'])['clock'].transform('nunique')
data2 = nl2[nl2['type'] == 3]
data1 = pl2.drop_duplicates(subset=columns, keep="first").reset_index()
data2 = data2[(data2['exchangeTime'] >= '13:00:00|500') & (data2['exchangeTime'] <= '15:00:00|0')]
test = pd.merge(data1, data2, left_on=columns, right_on=columns, how="outer")
n1 = test["clock_x"].count()
n2 = test["clock_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
test1 = test[~np.isnan(test["clock_x"])]
test1 = test1[~np.isnan(test1["clock_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
diff.describe([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')
print(sum((diff < 300) & (diff > -300))/len(diff))

109517
111283
114106
41.0
396.0
1230.0
22.0
55.0
128.0
0.5241813035409676
0.4723320899019626
0.6923913247230398


In [98]:
nl2[(nl2['exchangeTime'] == '13:00:04|500') & (nl2['secid'] == 'IC2008')]

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,exchangeTime,type,time
566859,12:59:37.013427,1595221177014869,1595221177014768,566860,2158,IC2008,1,6404.4,62027,6402.0,6402.4,6402.6,6402.8,6403.8,6404.6,6405.0,6405.4,6405.8,6406.0,1,2,5,2,1,1,9,1,2,2,NaN,76101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.894127e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:00:04|500,1,125937
566863,12:59:37.013427,1595221177014932,1595221177014780,566864,2158,IC2008,1,6404.4,62027,6402.0,6402.4,6402.6,6402.8,6403.8,6404.6,6405.0,6405.4,6405.8,6406.0,1,2,5,2,1,1,9,1,2,2,NaN,76101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.894127e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:00:04|500,3,125937


In [80]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

d1 = pd.read_csv(r'A:\check\20200722\fn3401\mdLog_20200722_0810.csv')
d11 = pd.read_csv(r'A:\check\20200722\fn3401\mdl2BestAndDeepLog_20200722_0810.csv')
corrections = {1088: 'IF2007',    
               1089: 'IF2008', 
               1090: 'IF2009', 
               1093: 'IF2012', 
               3363: 'IH2007',    
               3364: 'IH2008', 
               3365: 'IH2009', 
               3368: 'IH2012', 
               3463: 'IC2007',    
               3464: 'IC2008', 
               3465: 'IC2009', 
               3468: 'IC2012',  
               6263: 'XC2007',    
               6264: 'XC2008', 
               6265: 'XC2009', 
               6268: 'XC2012'}
d1.secid = d1.secid.map(corrections)
d11.secid = d11.secid.map(corrections)
d1 = d1[~d1['secid'].isnull()]
d11 = d11[~d11['secid'].isnull()]
d1 = d1[d1['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
d11 = d11[d11['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
display(d11.shape[0])
d11 = pd.merge(d11, d1.drop_duplicates(['secid', 'clockAtArrival', 'exchangeTime'])[['secid', 'clockAtArrival', 'exchangeTime']], on=['secid', 'clockAtArrival'], how='left')
display(d11.shape[0])
display(d11[d11['exchangeTime'].isnull()])
d11 = d11[~d11['lastPrice'].isnull()]

c1 = pd.read_csv(r'A:\check\20200722\fn3401\mdLog_20200722_1303.csv')
c11 = pd.read_csv(r'A:\check\20200722\fn3401\mdl2BestAndDeepLog_20200722_1303.csv')
corrections = {1088: 'IF2007',    
               1089: 'IF2008', 
               1090: 'IF2009', 
               1093: 'IF2012', 
               3363: 'IH2007',    
               3364: 'IH2008', 
               3365: 'IH2009', 
               3368: 'IH2012', 
               3463: 'IC2007',    
               3464: 'IC2008', 
               3465: 'IC2009', 
               3468: 'IC2012',  
               6263: 'XC2007',    
               6264: 'XC2008', 
               6265: 'XC2009', 
               6268: 'XC2012'}
c1.secid = c1.secid.map(corrections)
c11.secid = c11.secid.map(corrections)
c1 = c1[~c1['secid'].isnull()]
c11 = c11[~c11['secid'].isnull()]
c1 = c1[c1['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
c11 = c11[c11['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
display(c11.shape[0])
c11 = pd.merge(c11, c1.drop_duplicates(['secid', 'clockAtArrival', 'exchangeTime'])[['secid', 'clockAtArrival', 'exchangeTime']], on=['secid', 'clockAtArrival'], how='left')
display(c11.shape[0])
display(c11[c11['exchangeTime'].isnull()])
c11 = c11[~c11['lastPrice'].isnull()]

118256

118256

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,Unnamed: 65,exchangeTime


103642

103642

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,Unnamed: 65,exchangeTime


In [81]:
data1 = pd.concat([d11, c11])
data1.head(5)

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,Unnamed: 65,exchangeTime
1,2020-July-22 09:28:31.603662,1.595381e+15,1.595381e+15,36940.0,21098.0,IC2012,1.0,6240.0,34.0,6228.0,6230.0,6230.2,6232.2,6240.0,6245.0,6246.0,6247.0,6250.0,6255.0,1.0,2.0,1.0,1.0,4.0,1.0,2.0,1.0,21.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,42432000.0,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:29:00|200
2,2020-July-22 09:28:31.603736,1.595381e+15,1.595381e+15,36944.0,21098.0,IF2012,1.0,4592.6,7.0,4581.0,4582.0,4582.4,4583.0,4590.0,4592.8,4595.0,4598.0,4598.8,4599.8,5.0,1.0,1.0,1.0,7.0,2.0,3.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9644460.0,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:29:00|200
3,2020-July-22 09:28:31.603771,1.595381e+15,1.595381e+15,36946.0,21098.0,IH2012,1.0,3261.0,4.0,3250.0,3251.0,3251.2,3255.0,3258.0,3265.8,3266.2,3267.0,3267.4,3267.6,1.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3913200.0,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:29:00|200
4,2020-July-22 09:28:31.603916,1.595381e+15,1.595381e+15,36948.0,21098.0,IC2008,1.0,6469.0,222.0,6465.6,6466.4,6467.2,6468.0,6468.4,6469.2,6469.4,6469.6,6469.8,6470.0,10.0,4.0,1.0,1.0,1.0,8.0,34.0,80.0,42.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,287223600.0,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:29:00|200
5,2020-July-22 09:28:31.603988,1.595381e+15,1.595381e+15,36952.0,21098.0,IF2008,1.0,4657.8,203.0,4656.0,4657.0,4657.2,4657.4,4657.6,4657.8,4658.0,4658.2,4658.6,4658.8,3.0,8.0,5.0,14.0,2.0,1.0,4.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,283660020.0,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:29:00|200


In [82]:
data2 = pd.read_csv(r'A:\check\20200722\fn3402\mdLog_fut2_20200722_0909.csv')
data2.head(5)
data2 = data2.rename(columns={"secName":"secid"})
data2 = data2[data2['secid'].isin(['IC2009', 'IF2009', 'IH2009', 'IC2012', 'IF2012', 'IH2012',
       'IC2008', 'IF2008', 'IH2008'])]
data2 = data2.rename(columns={'genTime':"exchangeTime"})

In [83]:
data2.head(5)

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,exchangeTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10
0,09:29:10.781263,1595381350782754,1595381350782692,1,43258,IC2012,1,6240.0,34,6228.0,6230.0,6230.2,6232.2,6240.0,6245.0,6246.0,6247,6250,6255,1,2,1,1,4,1,2,1,21,2,09:29:00|200,47686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42432000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09:29:10.781263,1595381350782768,1595381350782720,2,43258,IF2012,1,4592.6,7,4581.0,4582.0,4582.4,4583.0,4590.0,4592.8,4595.0,4598,4598.8,4599.8,5,1,1,1,7,2,3,2,1,1,09:29:00|200,26683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9644460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:29:10.781263,1595381350782777,1595381350782729,3,43258,IC2012,1,6240.0,34,6228.0,6230.0,6230.2,6232.2,6240.0,6245.0,6246.0,6247,6250,6255,1,2,1,1,4,1,2,1,21,2,09:29:00|200,47686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42432000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09:29:10.781263,1595381350782786,1595381350782735,4,43258,IH2012,1,3261.0,4,3250.0,3251.0,3251.2,3255.0,3258.0,3265.8,3266.2,3267,3267.4,3267.6,1,3,2,2,1,1,1,1,1,1,09:29:00|200,14675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3913200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09:29:10.781263,1595381350782794,1595381350782742,5,43258,IF2012,1,4592.6,7,4581.0,4582.0,4582.4,4583.0,4590.0,4592.8,4595.0,4598,4598.8,4599.8,5,1,1,1,7,2,3,2,1,1,09:29:00|200,26683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9644460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
columns = ["secid", "lastPrice", "volume", "buyP5", "buyP4", "buyP3", "buyP2", "buyP1", "sellP1", "sellP2",
           "sellP3", "sellP4", "sellP5", "buyQ5", "buyQ4", "buyQ3", "buyQ2", "buyQ1", "sellQ1", "sellQ2", "sellQ3",
           "sellQ4", "sellQ5", 'turnover']
for cols in ['sellP3', 'sellP4', 'sellP5']:
    data2[cols] = data2[cols].astype(float)
data2 = data2[data2['volume'] > 0]
data1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
test = pd.merge(data1, data2, left_on=columns, right_on=columns, how="outer")
n1 = test["clock_x"].count()
n2 = test["clock_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)

211620
218674
218675


In [110]:
data1[(data1['secid'] == 'IC2008') & (data1['exchangeTime'] >= '11:29:59|200')]['clockAtArrival'].iloc[1]

1595388571328999.0

In [109]:
data2[(data2['secid'] == 'IC2008') & (data2['exchangeTime'] >= '11:29:59|200')].head(2)

,level_0,index,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,exchangeTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10
114072,118405,892636,11:30:08.078617,1595388608079616,1595388608079487,892637,43258,IC2008,1,6632.6,72399,6629.4,6629.6,6630.0,6630.4,6631.6,6632.0,6632.6,6632.8,6633.8,6634.4,2,1,2,2,3,1,4,15,1,1,11:29:59|200,77002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.523747e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114078,118412,892683,11:30:08.577617,1595388608578391,1595388608578291,892684,43258,IC2008,1,6631.6,72400,6629.4,6629.6,6630.0,6630.4,6631.6,6632.0,6632.6,6632.8,6633.8,6634.4,2,1,2,2,2,1,4,15,1,1,11:29:59|700,77001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.523880e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
pd.set_option('max_rows', 200)
test[test['clock_y'].isnull()].head(80)

,index_x,ms_x,clock_x,clockAtArrival_x,eventId_x,threadId_x,secid,isFaster_x,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi_x,suspension_x,buyQI5_x,buyQI4_x,buyQI3_x,buyQI2_x,buyQI1_x,sellQI1_x,sellQI2_x,sellQI3_x,sellQI4_x,sellQI5_x,tick_x,turnover,temp_x,buyP10_x,buyP9_x,buyP8_x,buyP7_x,buyP6_x,sellP6_x,sellP7_x,sellP8_x,sellP9_x,sellP10_x,buyQ10_x,buyQ9_x,buyQ8_x,buyQ7_x,buyQ6_x,sellQ6_x,sellQ7_x,sellQ8_x,sellQ9_x,sellQ10_x,Unnamed: 65,exchangeTime,index_y,ms_y,clock_y,clockAtArrival_y,eventId_y,threadId_y,isFaster_y,oi_y,suspension_y,buyQI5_y,buyQI4_y,buyQI3_y,buyQI2_y,buyQI1_y,sellQI1_y,sellQI2_y,sellQI3_y,sellQI4_y,sellQI5_y,tick_y,temp_y,buyP10_y,buyP9_y,buyP8_y,buyP7_y,buyP6_y,sellP6_y,sellP7_y,sellP8_y,sellP9_y,sellP10_y,buyQ10_y,buyQ9_y,buyQ8_y,buyQ7_y,buyQ6_y,sellQ6_y,sellQ7_y,sellQ8_y,sellQ9_y,sellQ10_y
2734,2760.0,2020-July-22 09:32:06.035125,1.595382e+15,1.595382e+15,60013.0,21098.0,IC2012,0.0,6217.4,541.0,6214.6,6215.0,6215.2,6216.2,6216.8,6219.2,6220.0,6221.6,6221.8,6222.0,1.0,1.0,1.0,2.0,1.0,3.0,7.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311.0,6.742708e+08,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:32:34|200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2735,2761.0,2020-July-22 09:32:06.035175,1.595382e+15,1.595382e+15,60017.0,21098.0,IF2012,0.0,4589.0,290.0,4585.0,4586.2,4588.4,4588.6,4589.2,4591.0,4591.2,4592.4,4596.4,4599.6,1.0,1.0,2.0,8.0,1.0,4.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,301.0,3.997298e+08,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:32:34|200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2737,2763.0,2020-July-22 09:32:06.035269,1.595382e+15,1.595382e+15,60021.0,21098.0,IC2008,0.0,6459.0,3683.0,6458.4,6458.6,6459.0,6460.0,6460.2,6461.6,6462.2,6462.8,6463.0,6464.4,2.0,2.0,2.0,20.0,1.0,3.0,3.0,1.0,3.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,315.0,4.765909e+09,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:32:34|200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2738,2764.0,2020-July-22 09:32:06.035307,1.595382e+15,1.595382e+15,60025.0,21098.0,IF2008,0.0,4653.2,3435.0,4651.4,4652.0,4652.8,4653.0,4653.2,4653.6,4653.8,4654.2,4654.4,4654.8,2.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,316.0,4.798501e+09,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:32:34|200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2739,2765.0,2020-July-22 09:32:06.035328,1.595382e+15,1.595382e+15,60027.0,21098.0,IH2008,0.0,3273.6,2138.0,3272.4,3272.6,3272.8,3273.0,3273.2,3274.6,3274.8,3275.0,3275.6,3275.8,1.0,2.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,315.0,2.101689e+09,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:32:34|200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2740,2766.0,2020-July-22 09:32:06.036035,1.595382e+15,1.595382e+15,60037.0,21098.0,IC2009,0.0,6392.0,957.0,6390.0,6390.2,6390.4,6391.0,6392.0,6392.2,6392.4,6392.6,6392.8,6393.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,N

In [104]:
data1[(data1['secid'] == 'IC2008') & (data1['exchangeTime'] >= '10:04:34|200')].head(10)[['clockAtArrival', 'secid', 'lastPrice', 'volume', 'exchangeTime']]

,clockAtArrival,secid,lastPrice,volume,exchangeTime
34561,1.595383e+15,IC2008,6557.8,26863.0,10:04:34|200
34568,1.595383e+15,IC2008,6557.8,26866.0,10:04:35|700
34578,1.595383e+15,IC2008,6558.4,26869.0,10:04:36|200
34587,1.595383e+15,IC2008,6556.8,26879.0,10:04:36|700
34595,1.595383e+15,IC2008,6557.0,26880.0,10:04:37|200
34602,1.595383e+15,IC2008,6556.4,26886.0,10:04:37|700
34609,1.595383e+15,IC2008,6556.0,26890.0,10:04:38|200
34615,1.595383e+15,IC2008,6556.8,26891.0,10:04:38|700
34621,1.595383e+15,IC2008,6554.6,26901.0,10:04:39|200
34627,1.595383e+15,IC2008,6557.0,26906.0,10:04:39|700


In [102]:
d11[d11['clockAtArrival'] == 1595383447143549.0]

,ms,clock,clockAtArrival,eventId,threadId,secid,isFaster,lastPrice,volume,buyP5,buyP4,buyP3,buyP2,buyP1,sellP1,sellP2,sellP3,sellP4,sellP5,buyQ5,buyQ4,buyQ3,buyQ2,buyQ1,sellQ1,sellQ2,sellQ3,sellQ4,sellQ5,genTime,oi,suspension,buyQI5,buyQI4,buyQI3,buyQI2,buyQI1,sellQI1,sellQI2,sellQI3,sellQI4,sellQI5,tick,turnover,temp,buyP10,buyP9,buyP8,buyP7,buyP6,sellP6,sellP7,sellP8,sellP9,sellP10,buyQ10,buyQ9,buyQ8,buyQ7,buyQ6,sellQ6,sellQ7,sellQ8,sellQ9,sellQ10,Unnamed: 65,exchangeTime
35316,2020-July-22 10:04:07.143566,1595383447143558,1595383447143549,292762,21098,IC2008,0,6557.8,26866,6556.0,6556.2,6556.4,6556.8,6557.0,6557.8,6558.2,6559.4,6560.0,6560.2,4.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,16.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4193.0,3.496691e+10,6.933350e-310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:04:35|700


In [103]:
data2[(data2['secid'] == 'IC2008') & (data2['exchangeTime'] >= '10:04:34|200')].head(10)[['clockAtArrival', 'secid', 'lastPrice', 'volume', 'exchangeTime']]

,clockAtArrival,secid,lastPrice,volume,exchangeTime
35333,1595383482807932,IC2008,6557.8,26863,10:04:34|200
35340,1595383483306273,IC2008,6557.8,26866,10:04:34|700
35348,1595383483806446,IC2008,6557.8,26866,10:04:35|200
35357,1595383484307864,IC2008,6557.8,26867,10:04:35|700
35365,1595383484807270,IC2008,6558.4,26869,10:04:36|200
35374,1595383485307457,IC2008,6556.8,26879,10:04:36|700
35383,1595383485806766,IC2008,6557.0,26880,10:04:37|200
35392,1595383486310239,IC2008,6556.4,26886,10:04:37|700
35400,1595383486807837,IC2008,6556.0,26890,10:04:38|200
35409,1595383487308083,IC2008,6556.8,26891,10:04:38|700


In [113]:
import json
with open("C:\Users\win\Downloads\ms.json",'r') as load_f:
    load_dict = json.load(load_f)
    print(load_dict)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-113-ae17ef5abf6f>, line 2)